## Neural Network

Here is the code for our neural network.

To load the information on the models we trained, run the following cell:

In [2]:
import numpy as np
from sklearn.neural_network import MLPClassifier
import pickle # to load information from trained models

models_file = open('models', 'rb')

models = pickle.load(models_file)

models_file.close()

`models` has the following structure:
```python
models = [
    model_info_a_log,  # models[0]
    model_info_b_log,  # models[1]
    model_info_c_log,  # models[2]
    model_info_d_log,  # models[3]
    
    model_info_a_tanh, # models[4]
    model_info_b_tanh, # models[5]
    model_info_c_tanh, # models[6]
    model_info_d_tanh  # models[7]
]
```

where each `model` index holds a nested dictionary of model information, keyed using the training and test accuracies:

```python
# structure of models[i]
models[i] = j : { # key is a number
                    'features' : name,
                    'layers' : layers,
                    'test size' : test_size,
                    'alpha' : alpha,
                    'lambda' : lam,
                    'train accuracy' : tr_acc,
                    'test accuracy' : ts_acc #,
                    # 'classifier' : clf # optional, adds ~2GB to models file so it was not saved 
                }

{
    # each key is a tuple of the training and testing accuracies
    (training_accuracy, testing_accuracy) : {
            # value keys are for name, layers, test size, alpha value, and lambda value
            'name' : name,
            'layers' : layers,
            'test size' : test_size,
            'alpha' : alpha,
            'lambda' : lam
        }
}
```

In [6]:
models

[{0: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 1000,
   'train accuracy': 0.30787037037037035,
   'test accuracy': 0.3151041666666667},
  1: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 100,
   'train accuracy': 0.34664351851851855,
   'test accuracy': 0.3133680555555556},
  2: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 1.0,
   'train accuracy': 0.33622685185185186,
   'test accuracy': 0.3289930555555556},
  3: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 0.1,
   'train accuracy': 0.3402777777777778,
   'test accuracy': 0.3229166666666667},
  4: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 0.01,
   'train accuracy': 0.33738425925925924,
   'test accura

In [8]:
for m in models:
    print(len(m))

576
576
576
576
576
576
576
576


=================================================================

There are 4 csv files corresponding to the different users' datasets:

In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df1 = pd.read_csv('./Dataset/user_a.csv', delimiter=',')
df2 = pd.read_csv('./Dataset/user_b.csv', delimiter=',')
df3 = pd.read_csv('./Dataset/user_c.csv', delimiter=',')
df4 = pd.read_csv('./Dataset/user_d.csv', delimiter=',')

In [2]:
from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import confusion_matrix, accuracy_score # 
import numpy as np
from sklearn.neural_network import MLPClassifier

# print training information -- accuracy, confusion matrix, sample predictions
def print_training_info(test_acc, train_acc, X_test, y_test, clf):
    print("* * *")
    print(f"training accuracy: {train_acc*100}%")
    print(f"testing accuracy: {test_acc*100}%")

    # show confusion matrix
    y_pred = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print (f"\nConfusion Matrix: \n{cm}\n")
    
    
    print(metrics.classification_report(y_test, y_pred, digits=3))

    print("\nfirst 10 test values and predictions:")

    # show predictions of first 10 test examples
    n = 10
    y_pred_n = clf.predict_proba(X_test[:n, :])
    for i in range(n):
        print(y_test[i], y_pred_n[i])

## Setting up for training

Below shows how we train the models. 

The `run` function trains a model according to its input specifications:

- It first splits our provided test data as specified using `test_size`.
- then it uses SKLearn's `MLPClassifier` to create a neural net with the desired number of layers, the max number of iterations, and the alpha and lambda values.
- it trains the neural net using the test data, then finds its accuracy on the training data and the test data.

In [21]:
from sklearn import metrics

def run(X, y, layers, test_size=0.4, lam=0.001, a=0.25, iter_num=3000, batch_size='auto', act='logistic', threshold=0.8):
    X_scale = StandardScaler()
    X = X_scale.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    clf = MLPClassifier(activation=act,
                        hidden_layer_sizes=layers, 
                        alpha=lam, # refers to our lambda
                        learning_rate_init=a, # refers to our alpha
                        max_iter=iter_num,
                        batch_size=batch_size, # default is 200
                        solver='sgd', 
                        random_state=1,
                        verbose=False
                       )
    clf.fit(X_train, y_train)
    
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)

    # print accuracy, confusion matrix, sample predictions above threshold
    if test_acc > threshold:
        print_training_info(test_acc, train_acc, X_test, y_test, clf)
        
    return train_acc, test_acc, clf

the `automate_run` function calls run many times using different specifications. Here is a list of the things `automate_run` varies so far:
- the features used (all, just the stds, just the means)
- the number of hidden layers and their structure
- the activation function (logistic or tanh)
- the training size/test size
- the alpha values ($\alpha \in [0.001, 1]$)
- the lambda values ($\lambda \in [0.00001, 1000]$)


In [4]:
# goes through many different ways to train a model
# def automate_run(df, dic, act='logistic'):
#     i = 0
#     test_size = 0.4
#     for name, X, y in (('all features', df.values[:, 1:], df.values[:, 0].astype(int)), 
#                        ('std features', df.values[:, 1::2], df.values[:, 0].astype(int)), 
#                        ('mean features', df.values[:, 2::2], df.values[:, 0].astype(int))):
#         for layers in ((112, 56, 30, 10),
#                        (112),
#                        (112, 112, 112)):
#             for alpha in (1., 0.1, 0.01, 0.001):
#                 for lam in (1000, 100, 1., 0.1, 0.01, 0.001, 0.0001, 0.00001):
#                     print('\n==============================\n')
#                     print(f'{name}:\nlayers={layers}\ntest_size={test_size}\nalpha={alpha}\nlambda={lam}\n')
#                     tr_acc, ts_acc, clf = run(X, y, layers, test_size, lam, alpha, act=act)
#                     dic[i] = {
#                         'features' : name,
#                         'layers' : layers,
#                         'test size' : test_size,
#                         'alpha' : alpha,
#                         'lambda' : lam,
#                         'train accuracy' : tr_acc,
#                         'test accuracy' : ts_acc,
#                         'classifier' : clf
#                     }
#                     i += 1

In [5]:
# goes through many different ways to train a model
def automate_run(df, dic, act='logistic'):
    i = 0
    test_size = 0.4
    for name, X, y in [('all features', df.values[:, 1:], df.values[:, 0].astype(int))]:
        for layers in ([112],
                       [112,112,112],
                       [112, 56, 30, 10]):
            for alpha in (1., 0.1, 0.01, 0.001):
                for lam in (1000, 100, 1., 0.1, 0.01, 0.001, 0.0001, 0.00001):
                    print('\n==============================\n')
                    print(f'{name}:\nlayers={layers}\ntest_size={test_size}\nalpha={alpha}\nlambda={lam}\n')
                    tr_acc, ts_acc, cm = run(X, y, layers, test_size, lam, alpha, act=act)
                    dic[i] = {
                        'features' : name,
                        'layers' : layers,
                        'test size' : test_size,
                        'alpha' : alpha,
                        'lambda' : lam,
                        'train accuracy' : tr_acc,
                        'test accuracy' : ts_acc,
#                         'confusion matrix' : cm
                    }
                    i += 1

In [6]:
# prints the model accuracies and info above the given accuracy threshold
def show_above(d, accuracy=0.85):
    to_print = []
    for key, info in d.items():
        if info['test size'] == 0.4 and info['test accuracy'] > accuracy:
            to_print.append((info['test accuracy'], key, info))

    for acc, key, info in sorted(to_print, reverse=True):
        print(acc)
        for k, v in info.items():
            print(f'\t{k}: {v}')

## Training the models on different users

### User A

In [12]:
model_info_a_log = {} # key is just an int, values are info
automate_run(df1, model_info_a_log)



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
l

* * *
training accuracy: 100.0%
testing accuracy: 81.25%

Confusion Matrix: 
[[308  48  25]
 [ 35 289  42]
 [ 21  45 339]]

first 10 test values and predictions:
2 [5.59632257e-12 2.81061611e-04 9.99718938e-01]
1 [9.71922081e-01 2.73190701e-02 7.58849159e-04]
1 [0.00243584 0.72025404 0.27731013]
2 [0.04702062 0.86324477 0.08973461]
2 [1.23427584e-03 2.87277650e-04 9.98478447e-01]
0 [9.42198118e-01 5.75056082e-02 2.96274147e-04]
2 [5.05479900e-07 7.31945082e-04 9.99267549e-01]
1 [0.00486078 0.78392056 0.21121866]
0 [9.99999970e-01 3.00095726e-08 7.93475759e-12]
0 [9.99719298e-01 9.26453684e-08 2.80609003e-04]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 80.55555555555556%

Confusion Matrix: 
[[312  33  35]
 [ 33 328  34]
 [ 28  61 288]]

first 10 test values and predictions:
1 [6.97603841e-04 9.99301382e-01 1.01427599e-06]
1 [9.32755676e-03 9.90668856e-01 3.58725373e-06]
0 [0.79465419 0.00325112 0.20209469]
0 [9.9999

* * *
training accuracy: 100.0%
testing accuracy: 81.59722222222221%

Confusion Matrix: 
[[311  51  30]
 [ 21 315  49]
 [ 19  42 314]]

first 10 test values and predictions:
1 [0.01363517 0.97671179 0.00965305]
0 [0.95307772 0.04472091 0.00220137]
1 [2.02544702e-07 9.99999627e-01 1.70031333e-07]
0 [9.97865976e-01 6.64169753e-04 1.46985378e-03]
1 [0.00751149 0.97717652 0.015312  ]
2 [0.00143423 0.045116   0.95344978]
0 [9.99386903e-01 3.56577793e-04 2.56519210e-04]
1 [6.71921829e-04 9.99317993e-01 1.00848407e-05]
0 [9.97488401e-01 2.33460746e-03 1.76991146e-04]
0 [0.98376901 0.01408783 0.00214317]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 80.03472222222221%

Confusion Matrix: 
[[312  26  32]
 [ 37 299  42]
 [ 27  66 311]]

first 10 test values and predictions:
0 [9.79867121e-01 6.21848586e-04 1.95110300e-02]
2 [0.0135557  0.00220502 0.98423927]
1 [0.0063559  0.99045678 0.00318732]
0 [0.81990366 0.17404581 0.006050

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=112
test_size=0.4
alpha=0.001
lambda=0.01



...



std features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.0001



std features:
layers=112
test_size=0.3
alpha=0.001
lambda=1e-05



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1000



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=100



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1.0



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.1



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.01



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.001



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.0001



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1e-05



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1000



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=100



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1.0



std features:
layers=(112, 1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.01



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.0001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1e-05



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1000



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1.0



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=0.1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=0.01



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=0.001



mean features:
layers=(112, 56, 30, 10)
test



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.1



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.01



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.001



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.0001



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=1e-05



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=1000



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=1.0



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.1



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.01

* * *
training accuracy: 99.82638888888889%
testing accuracy: 80.03472222222221%

Confusion Matrix: 
[[317  41  27]
 [ 37 295  48]
 [ 41  36 310]]

first 10 test values and predictions:
1 [6.51484133e-05 9.90424686e-01 9.51016562e-03]
0 [9.99493329e-01 4.95842877e-04 1.08285316e-05]
0 [9.82699586e-01 1.66676454e-04 1.71337374e-02]
1 [3.2461807

* * *
training accuracy: 99.95039682539682%
testing accuracy: 82.63888888888889%

Confusion Matrix: 
[[266  20  13]
 [ 27 217  40]
 [ 20  30 231]]

first 10 test values and predictions:
0 [9.99944106e-01 1.57839103e-05 4.01100570e-05]
0 [0.78669287 0.20637908 0.00692805]
0 [0.90028709 0.06725138 0.03246152]
0 [0.07101405 0.00094171 0.92804424]
1 [0.07625464 0.83375265 0.08999271]
1 [7.60444055e-03 9.92089457e-01 3.06102591e-04]
1 [8.94666960e-04 9.96267732e-01 2.83760081e-03]
1 [0.02923693 0.01670729 0.95405579]
0 [9.80138742e-01 2.46594358e-04 1.96146638e-02]
2 [0.01904407 0.92260815 0.05834778]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=1e-05



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=1000



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=100



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=1.0



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.1



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.0



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1e-05



In [11]:
# looking at the top models
model_info_a_log = models[0]
show_above(model_info_a_log, 0.85)

0.8515625
	features: all features
	layers: 112
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8515625


In [21]:
# Using tanh
model_info_a_tanh = {}
automate_run(df1, model_info_a_tanh, act='tanh')



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 99.65277777777779%
testing accuracy: 83.76736111111111%

Confusion Matrix: 
[[315  25  19]
 [ 34 317  43]
 [ 23  43 33



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=

* * *
training accuracy: 100.0%
testing accuracy: 82.06018518518519%

Confusion Matrix: 
[[250  17  24]
 [ 26 240  35]
 [ 18  35 219]]

first 10 test values and predictions:
2 [0.00605567 0.11202648 0.88191785]
0 [9.99785769e-01 3.87222566e-05 1.75508263e-04]
1 [2.20377603e-05 9.99839362e-01 1.38600003e-04]
1 [0.01071982 0.98434247 0.00493772]
0 [0.70208764 0.27484057 0.02307178]
1 [0.00827928 0.73491631 0.25680441]
0 [9.97034206e-01 2.94411713e-03 2.16772274e-05]
1 [0.00814826 0.99054644 0.0013053 ]
0 [0.62513665 0.07189757 0.30296578]
1 [1.39657378e-03 9.97638936e-01 9.64490154e-04]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.01
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 85.18518518518519%

Confusion Matrix: 
[

* * *
training accuracy: 100.0%
testing accuracy: 81.42361111111111%

Confusion Matrix: 
[[304  38  25]
 [ 32 313  49]
 [ 21  49 321]]

first 10 test values and predictions:
1 [1.70164368e-03 9.98289482e-01 8.87453165e-06]
2 [1.17092852e-04 8.54852112e-04 9.99028055e-01]
2 [8.02248762e-02 7.21725537e-04 9.19053398e-01]
0 [9.98871475e-01 3.24388330e-04 8.04136171e-04]
1 [0.06900206 0.19065321 0.74034473]
2 [7.93228273e-06 6.77738892e-05 9.99924294e-01]
1 [6.43437078e-06 9.99779959e-01 2.13606470e-04]
0 [9.99829215e-01 6.33566953e-05 1.07428406e-04]
1 [5.59915620e-04 9.93602174e-01 5.83791052e-03]
2 [0.23254925 0.00250261 0.76494815]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1000



all features:
layers=112
test_size=0.4
alpha=0.01
lambda=100



all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1.0

* * *
training accuracy: 99.82638888888889%
testing accuracy: 84.375%

Confusion Matrix: 
[[328  36  17]
 [ 23 313  45]
 [ 21  38 331]]

first 10 test values and predi



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.1



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.01



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.001



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.0001



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=1e-05



all features:
layers=112
test_size=0.3
alpha=0.1
lambda=1000



all features:
layers=112
test_size=0.3
alpha=0.1
lambda=100



all features:
layers=112
test_size=0.3
alpha=0.1
lambda=1.0

* * *
training accuracy: 92.26190476190477%
testing accuracy: 83.2175925925926%

Confusion Matrix: 
[[268  19  11]
 [ 32 234  27]
 [ 31  25 217]]

first 10 test values and predictions:
1 [0.05746494 0.87111273 0.07142234]
0 [0.83741362 0.06651186 0.09607452]
1 [0.00134144 0.97517647 0.0234821 ]
0 [0.58385531 0.27517697 0.14096772]
0 [0.72554111 0.16137459 0.1130843 ]
1 [0.11636267 0.88248785 0.00114948]
0 [9.80929169e-01 1.85066020e-02 5.64229012e-04]
1 [0.02213023 0.77522407 0.2026

...



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.01
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 83.44907407407408%

Confusion Matrix: 
[[240  20  19]
 [ 23 231  32]
 [ 17  32 250]]

first 10 test values and predictions:
0 [9.99904068e-01 1.03854336e-05 8.55464937e-05]
1 [0.00155936 0.05722796 0.94121268]
1 [4.51097765e-06 9.99180358e-01 8.15131321e-04]
1 [9.98832490e-01 1.10313687e-03 6.43735681e-05]
0 [0.36012035 0.35824759 0.28163206]
1 [5.74713824e-06 9.99669264e-01 3.24988837e-04]
1 [3.61358636e-04 9.99637341e-01 1.30062576e-06]
1 [9.98702806e-01 4.52386719e-04 8.44807570e-04]
0 [0.94173008 0.00645509 0.05181484]
1 [1.08127511e-05 9.98347285e-01 1.64190220e-03]


mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 81.82870370370371%

Confusion Matrix: 
[[246  22  18]
 [ 35 238  21]
 [ 24  37 223]]

first 10 test values and predictions:
2 [4.77301533e-04 2.63728654e-02 9.73

* * *
training accuracy: 100.0%
testing accuracy: 80.12152777777779%

Confusion Matrix: 
[[316  32  42]
 [ 36 312  44]
 [ 24  51 295]]

first 10 test values and predictions:
0 [0.212883   0.77530707 0.01180993]
1 [7.46952915e-06 9.99928920e-01 6.36105516e-05]
1 [0.02596672 0.96985942 0.00417386]
1 [0.33860574 0.66053121 0.00086305]
2 [0.00652976 0.00466215 0.98880809]
1 [0.38346653 0.57974331 0.03679016]
1 [1.58507580e-03 9.97562450e-01 8.52474359e-04]
1 [0.13174179 0.85469341 0.0135648 ]
2 [0.13665712 0.55288216 0.31046072]
0 [0.00991566 0.98676025 0.00332409]


mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.01



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 82.8125%

Confusion Matrix: 
[[322  29  30]
 [ 40 313  35]
 [ 27  37 319]]

first 10 test values and predictions:
1 [0.07414565 0.92005535 0.005799  ]
2 [3.54585950e-05 8.93866511e-02 9.10577890e-01]
2 [1.04998875e-05 5.26411511e-01 4.73577989e-01]




mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=1.0



mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.1

* * *
training accuracy: 99.95039682539682%
testing accuracy: 81.59722222222221%

Confusion Matrix: 
[[263  26  24]
 [ 21 217  36]
 [ 11  41 225]]

first 10 test values and predictions:
0 [0.91099247 0.07900924 0.00999829]
2 [4.12046738e-03 5.30274241e-05 9.95826505e-01]
0 [0.66817634 0.31663113 0.01519253]
0 [0.92555686 0.02642314 0.04802001]
0 [9.94189362e-01 5.23557819e-03 5.75060131e-04]
2 [0.1380524  0.02971366 0.83223394]
2 [0.00520597 0.02270771 0.97208633]
1 [3.72411169e-04 8.01815622e-01 1.97811966e-01]
1 [0.11597599 0.64359161 0.2404324 ]
2 [4.30609740e-04 1.56222222e-05 9.99553768e-01]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 80.55555555555556%

Confusion Matrix: 
[[252  19  31]
 [ 27 228  44]
 [ 17  30 216]]

first 10 test values and predictions:
1 [1.37676295e-03 9.986121

* * *
training accuracy: 100.0%
testing accuracy: 80.55555555555556%

Confusion Matrix: 
[[316  25  35]
 [ 37 304  49]
 [ 30  48 308]]

first 10 test values and predictions:
2 [0.15149561 0.08805863 0.76044577]
0 [9.99601434e-01 3.28874006e-04 6.96921376e-05]
2 [8.90226636e-06 4.84563714e-03 9.95145461e-01]
1 [0.0739252  0.0043049  0.92176989]
0 [9.9595820e-01 3.5466592e-06 4.0382534e-03]
1 [6.92315950e-04 8.98302897e-01 1.01004787e-01]
1 [0.29964125 0.46883846 0.23152029]
2 [2.66272212e-07 9.99945791e-01 5.39427861e-05]
1 [4.57889219e-04 9.99518092e-01 2.40184418e-05]
0 [9.99005775e-01 2.41721418e-06 9.91807470e-04]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[325  34  26]
 [ 31 311  47]
 [ 24  49 305]]

first 10 test values and predictions:
0 [9.96168985e-01 3.83098332e-03 3.13909156e-08]
0 [9.99998943e-01 1.05727545e-06 2.73488134e-11]
0 [9.99216608e-01 1.46230



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=1.0

* * *
training accuracy: 98.61111111111111%
testing accuracy: 80.78703703703704%

Confusion Matrix: 
[[245  32  21]
 [ 23 225  38]
 [ 18  34 228]]

first 10 test values and predictions:
1 [1.00852393e-06 9.98963765e-01 1.03522613e-03]
0 [9.96295738e-01 3.62231152e-03 8.19505540e-05]
1 [1.23621751e-05 9.99185508e-01 8.02130271e-04]
2 [0.03735363 0.13424292 0.82840345]
0 [5.17389943e-01 4.82554606e-01 5.54510475e-05]
1 [0.00142745 0.87266987 0.12590269]
2 [0.00947271 0.26475446 0.72577283]
1 [0.04944855 0.91944317 0.03110828]
1 [3.42584980e-04 9.60745104e-01 3.89123115e-02]
0 [0.60986504 0.01591158 0.37422338]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 82.87037037037037%

Confusion Matrix: 
[[264  19  18]
 [ 24 236  25]
 [ 22  40 216]]

first 10 test val

In [12]:
model_info_a_tanh = models[4]
show_above(model_info_a_tanh, 0.85)

0.8680555555555556
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 0.9994212962962963
	test accuracy: 0.8680555555555556
0.8532986111111112
	features: all features
	layers: (112, 56, 30, 10)
	test size: 0.4
	alpha: 0.01
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.8532986111111112


### User B

In [23]:
model_info_b_log = {} # key is just an int, values are info
automate_run(df2, model_info_b_log)



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
l

* * *
training accuracy: 100.0%
testing accuracy: 86.45833333333334%

Confusion Matrix: 
[[346  32  28]
 [ 22 314  22]
 [ 25  27 336]]

first 10 test values and predictions:
2 [0.59326601 0.00398405 0.40274993]
1 [0.19864151 0.79540979 0.0059487 ]
0 [9.96199826e-01 2.96602887e-03 8.34144723e-04]
1 [4.25044106e-05 9.99955467e-01 2.02888546e-06]
0 [9.99902994e-01 1.70572294e-08 9.69884567e-05]
2 [1.44017390e-04 1.08653910e-01 8.91202073e-01]
1 [7.25335972e-05 9.99926813e-01 6.53318728e-07]
2 [1.20235631e-07 4.50676157e-03 9.95493118e-01]
2 [9.60083565e-11 1.22045608e-06 9.99998779e-01]
1 [2.52581336e-07 5.46523863e-03 9.94534509e-01]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 84.72222222222221%

Confusion Matrix: 
[[357  30  18]
 [ 24 307  35]
 [ 34  35 312]]

first 10 test values and predictions:
0 [9.99959639e-01 2.07007645e-07 4.01538629e-05]
2 [3.90123828e-04 3.21450951e-03 9.96395367e-01]
1 [8.24097790e-06 3.243

* * *
training accuracy: 100.0%
testing accuracy: 85.85069444444444%

Confusion Matrix: 
[[337  24  19]
 [ 31 327  35]
 [ 17  37 325]]

first 10 test values and predictions:
2 [0.16346644 0.03112461 0.80540895]
0 [0.97830869 0.01185572 0.00983559]
0 [0.99362974 0.00445821 0.00191205]
2 [2.18428556e-02 5.04988825e-04 9.77652156e-01]
2 [0.00391277 0.35959323 0.636494  ]
1 [0.00184928 0.99715219 0.00099853]
2 [0.5316432  0.35334646 0.11501034]
1 [9.05131531e-06 9.94308975e-01 5.68197372e-03]
0 [9.97621147e-01 3.31238735e-04 2.04761395e-03]
1 [0.51764401 0.12642718 0.35592881]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 84.02777777777779%

Confusion Matrix: 
[[315  35  33]
 [ 20 325  32]
 [ 22  42 328]]

first 10 test values and predictions:
2 [0.16355084 0.01225776 0.8241914 ]
1 [6.68888375e-03 9.93197959e-01 1.13157105e-04]
0 [0.99223142 0.00384337 0.00392521]
0 [0.95902833 0.02330681 0.01766486]
1 [0.00144098 0.995

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=112
test_size=0.4
alpha=0.001
lambda=0.01



...



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1e-05



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1000



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=100



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1.0



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.01



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.0001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1e-05



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1000



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=100



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1.0



mean features:
layers=(112, 56, 3



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=100



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=1.0



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 84.98263888888889%

Confusion Matrix: 
[[360  21  19]
 [ 37 306  36]
 [ 28  32 313]]

first 10 test values and predictions:
1 [0.87321601 0.12406649 0.0027175 ]
0 [0.84301697 0.1268037  0.03017933]
0 [0.9062477  0.04279392 0.05095838]
0 [0.97006805 0.02520702 0.00472493]
1 [0.17891672 0.40813223 0.41295104]
1 [0.77966145 0.20772992 0.01260863]
0 [0.9845243  0.00123822 0.01423749]
2 [0.05679047 0.25352933 0.6896802 ]
0 [0.74853441 0.24638228 0.00508331]
1 [0.12362865 0.87357003 0.00280132]


mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 85.24305555555556%

Confusion Matrix: 
[[327  34  17]
 [ 29 329  41]
 [ 16  33 326]]

first 10 test values and predictions:
2 [0.08442385 0.37931851 



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=1000



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=100



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=1.0



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.1

* * *
training accuracy: 95.68452380952381%
testing accuracy: 81.82870370370371%

Confusion Matrix: 
[[254  13  13]
 [ 36 239  32]
 [ 23  40 214]]

first 10 test values and predictions:
1 [0.0169847  0.98185407 0.00116124]
2 [0.69226436 0.0381633  0.26957234]
1 [0.23112621 0.13256616 0.63630764]
1 [0.09185648 0.90256314 0.00558037]
0 [9.89040435e-01 7.32826505e-05 1.08862825e-02]
0 [0.96026204 0.02929388 0.01044408]
0 [0.81805759 0.08245516 0.09948725]
2 [3.93602306e-06 1.63423259e-05 9.99979722e-01]
0 [0.98105755 0.00270412 0.01623833]
2 [7.38735528e-04 1.30818558e-01 8.68442706e-01]


mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 84.72222222222221%

Confusion Matrix

* * *
training accuracy: 100.0%
testing accuracy: 82.87037037037037%

Confusion Matrix: 
[[246  25  23]
 [ 20 228  27]
 [ 21  32 242]]

first 10 test values and predictions:
2 [2.28730119e-04 5.48857583e-01 4.50913687e-01]
0 [9.98477829e-01 1.21783980e-03 3.04331198e-04]
2 [5.11526595e-10 8.68588560e-09 9.99999991e-01]
1 [0.00340511 0.9920346  0.00456028]
1 [3.88233775e-04 9.90708221e-01 8.90354564e-03]
2 [2.41311381e-06 5.50953232e-04 9.99446634e-01]
2 [0.78366846 0.00214791 0.21418363]
1 [0.00626853 0.98314278 0.01058868]
0 [0.48072572 0.40618758 0.11308671]
0 [0.80001661 0.0098698  0.19011359]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 85.06944444444444%

Confusion Matrix: 
[[248  17  13]
 [ 30 246  25]
 [ 16  28 241]]

first 10 test values and predictions:
1 [0.40686513 0.58959692 0.00353794]
1 [1.86890685e-07 9.99954028e-01 4.57847599e-05]
0 [0.99710214 0.00151957 0.00137829]
2 [0.00273693 0.02506375 0.97219



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1e-05



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0

In [13]:
# looking at the top models
model_info_b_log = models[1]
show_above(model_info_b_log, 0.85)

0.8862847222222222
	features: all features
	layers: (112, 112, 112)
	test size: 0.4
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8862847222222222
0.8836805555555556
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8836805555555556
0.8802083333333334
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.1
	lambda: 0.001
	train accuracy: 1.0
	test accuracy: 0.8802083333333334
0.8784722222222222
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.01
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8784722222222222
0.8758680555555556
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.01
	lambda: 1e-05
	train accuracy: 1.0
	test accuracy: 0.8758680555555556
0.8758680555555556
	features: all features
	layers: 112
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8758680555555556
0.8741319444444444
	features: all features
	layers: 112
	test si

In [25]:
model_info_b_tanh = {} # key is just an int, values are info
automate_run(df2, model_info_b_tanh, act='tanh')



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 90.27777777777779%

Confusion Matrix: 
[[373  18  16]
 [ 17 340  24]
 [ 13  24 327]]

first 1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[331  22  19]
 [ 34 311  49]
 [ 40  47 299]]

first 10 test values and predictions:
0 [9.96630529e-01 2.42212373e-03 9.47347735e-04]
2 [4.73553351e-05 1.36871394e-04 9.99815773e-01]
1 [4.36740216e-04 9.97025418e-01 2.53784206e-03]
2 [0.00392979 0.02666674 0.96940346]
0 [9.92699156e-01 7.12264055e-03 1.78203423e-04]
1 [7.63422453e-05 9.97487014e-01 2.43664372e-03]
2 [0.07376628 0.00201488 0.92421884]
2 [6.77134410e-05 2.94066664e-03 9.96991620e-01]
2 [0.00332735 0.00407399 0.99259866]
1 [0.98936194 0.00882934 0.00180872]


all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 85.5902777777777

* * *
training accuracy: 100.0%
testing accuracy: 85.30092592592592%

Confusion Matrix: 
[[246  16  22]
 [ 21 233  26]
 [ 20  22 258]]

first 10 test values and predictions:
2 [4.27455423e-05 2.73706107e-04 9.99683548e-01]
1 [3.04394988e-02 9.69500775e-01 5.97263078e-05]
2 [5.02131895e-06 5.41416295e-02 9.45853349e-01]
0 [2.03138188e-04 9.77870042e-01 2.19268202e-02]
2 [5.74348079e-05 6.75670685e-05 9.99874998e-01]
0 [1.54801325e-02 9.84228536e-01 2.91331804e-04]
0 [0.70538667 0.29215924 0.00245409]
1 [3.86869634e-05 9.99769630e-01 1.91682696e-04]
2 [8.31137688e-03 9.28832101e-04 9.90759791e-01]
2 [3.46438129e-04 1.90224221e-04 9.99463338e-01]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 86.57407407407408%

Confusion Matrix: 
[[263  21  14]
 [ 21 244  24]
 [  7  29 241]]

first 10 test values and predictions:
1 [5.76582374e-05 9.99692502e-01 2.49840044e-04]
2 [6.89348069e-05 1.99472412e-03 9.97936341e-0

* * *
training accuracy: 100.0%
testing accuracy: 82.87037037037037%

Confusion Matrix: 
[[247  21  19]
 [ 20 246  28]
 [ 22  38 223]]

first 10 test values and predictions:
0 [9.85947739e-01 1.35521767e-04 1.39167389e-02]
1 [7.25491225e-04 9.94440618e-01 4.83389064e-03]
2 [8.03911145e-05 8.55913576e-04 9.99063695e-01]
1 [1.49582341e-03 9.97563794e-01 9.40382608e-04]
0 [9.48531271e-01 5.11228251e-02 3.45904078e-04]
1 [0.00291061 0.15572591 0.84136348]
0 [9.98848616e-01 2.56553301e-04 8.94830538e-04]
2 [5.36490447e-04 8.35272138e-04 9.98628237e-01]
0 [0.08683396 0.89561857 0.01754748]
1 [0.00236951 0.99618147 0.00144902]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 82.4074074074074%

Confusion Matrix: 
[[236  22  18]
 [ 21 229  35]
 [ 21  35 247]]

first 10 test values and predictions:
2 [0.00154475 0.16028724 0.83816801]
1 [0.01889526 0.05651986 0.92458488]
0 [9.91575979e-01 8.00792051e-03 4.16100614

* * *
training accuracy: 100.0%
testing accuracy: 86.37152777777779%

Confusion Matrix: 
[[325  20  20]
 [ 25 324  35]
 [ 20  37 346]]

first 10 test values and predictions:
0 [3.21695582e-02 9.67755178e-01 7.52634775e-05]
0 [9.99996743e-01 3.20180146e-06 5.56385725e-08]
2 [1.67502141e-04 9.82908309e-04 9.98849590e-01]
1 [6.62330253e-10 9.99995985e-01 4.01471501e-06]
2 [3.83827032e-05 5.11399422e-02 9.48821675e-01]
0 [9.98211291e-01 1.03922953e-03 7.49479796e-04]
0 [9.99934670e-01 3.21238372e-06 6.21172701e-05]
2 [0.26209012 0.01443267 0.72347721]
1 [1.10044083e-04 1.02750463e-03 9.98862451e-01]
2 [0.00349328 0.00655541 0.98995131]


all features:
layers=112
test_size=0.4
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 82.29166666666666%

Confusion Matrix: 
[[333  35  28]
 [ 36 294  38]
 [ 23  44 321]]

first 10 test values and predictions:
0 [9.90445134e-01 9.55484939e-03 1.62464876e-08]
1 [2.08855996e-04 9.47871433e-01 5.19197111e-02]
0 [9.93554616e-01 6.007

...



mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=1.0

* * *
training accuracy: 94.29563492063492%
testing accuracy: 84.49074074074075%

Confusion Matrix: 
[[261  22  15]
 [ 24 238  19]
 [ 13  41 231]]

first 10 test values and predictions:
2 [0.03056015 0.40645611 0.56298374]
2 [0.10188771 0.24455678 0.65355552]
2 [0.03706002 0.01783718 0.94510279]
0 [0.86843863 0.02712381 0.10443756]
0 [0.91675382 0.06997431 0.01327187]
2 [0.0379068  0.39350035 0.56859285]
2 [0.42288607 0.08426243 0.4928515 ]
0 [0.36181434 0.63230822 0.00587744]
2 [0.00678161 0.02787171 0.96534668]
2 [0.0786506  0.18170163 0.73964777]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 88.19444444444444%

Confusion Matrix: 
[[258  19  17]
 [ 12 239  15]
 [ 14  25 265]]

first 10 test values and predictions:
0 [0.95334583 0.04008108 0.00657309]
2 [2.96452322e-04 2.60923307e-02 9.73611217e-01]
1 [1.59608863e-03 9.98159858e-01 2.44053191e-04]
0 [9.



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1e-05



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 86.89236111111111%

Confusion Matrix: 
[[338  23  32]
 [ 20 321  20]
 [ 18  38 342]]

first 10 test values and predictions:
0 [9.98925939e-01 1.01909328e-03 5.49672708e-05]
1 [1.527

* * *
training accuracy: 100.0%
testing accuracy: 85.41666666666666%

Confusion Matrix: 
[[315  26  23]
 [ 28 335  36]
 [ 34  21 334]]

first 10 test values and predictions:
0 [9.99997946e-01 2.62932703e-11 2.05382148e-06]
0 [9.99995088e-01 1.99478324e-08 4.89216205e-06]
2 [3.52107152e-08 3.32174557e-06 9.99996643e-01]
0 [9.68533640e-01 3.14657038e-02 6.56205005e-07]
1 [0.41592276 0.56508113 0.01899611]
0 [9.99960317e-01 3.88366577e-05 8.46363768e-07]
0 [9.99692811e-01 3.06226983e-04 9.62378678e-07]
1 [0.03949471 0.95925607 0.00124922]
2 [4.55879920e-04 4.78287594e-06 9.99539337e-01]
2 [5.15063003e-04 1.81739052e-03 9.97667546e-01]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 83.68055555555556%

Confusion Matrix: 
[[326  40  33]
 [ 21 312  36]
 [ 20  38 326]]

first 10 test values and predictions:
1 [0.00052947 0.50065436 0.49881617]
1 [1.48822520e-07 9.93627640e-01 6.37221078e-03]
0 [0.88300626 0.06500

* * *
training accuracy: 100.0%
testing accuracy: 87.84722222222221%

Confusion Matrix: 
[[262  14  11]
 [ 16 254  26]
 [ 17  21 243]]

first 10 test values and predictions:
2 [1.26407526e-08 1.31826676e-04 9.99868161e-01]
2 [7.50821647e-05 5.23417942e-06 9.99919684e-01]
1 [1.31836304e-04 9.99866549e-01 1.61502521e-06]
2 [2.15050713e-04 1.58298163e-02 9.83955133e-01]
2 [4.57256460e-02 1.82184158e-05 9.54256136e-01]
2 [1.44476266e-04 3.98284090e-01 6.01571434e-01]
0 [2.03379101e-02 1.85730834e-06 9.79660233e-01]
1 [1.03617950e-03 9.98963820e-01 7.83845508e-10]
0 [9.98581999e-01 1.33690098e-03 8.10996989e-05]
1 [1.82639425e-10 9.99999805e-01 1.94831078e-07]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 86.92129629629629%

Confusion Matrix: 
[[261  21  16]
 [ 18 248  18]
 [ 13  27 242]]

first 10 test values and predictions:
0 [9.99999071e-01 9.00361565e-07 2.88462475e-08]
1 [3.13684086e-04 9.98836323e-01 8.

* * *
training accuracy: 100.0%
testing accuracy: 85.41666666666666%

Confusion Matrix: 
[[246  24  19]
 [ 17 229  22]
 [ 20  24 263]]

first 10 test values and predictions:
0 [9.93862705e-01 6.13161267e-03 5.68260603e-06]
0 [9.99791475e-01 2.06397859e-04 2.12700318e-06]
1 [1.20012191e-05 9.57546257e-01 4.24417417e-02]
0 [0.49518267 0.49794501 0.00687233]
2 [1.16443948e-03 5.98921481e-07 9.98834962e-01]
0 [0.72929978 0.01142651 0.2592737 ]
2 [2.40527192e-04 6.07852683e-02 9.38974205e-01]
2 [6.14900818e-09 8.42220750e-07 9.99999152e-01]
2 [1.89274386e-04 2.26529658e-01 7.73281068e-01]
1 [2.35238024e-05 9.95500444e-01 4.47603250e-03]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 83.33333333333334%

Confusion Matrix: 
[[255  27  18]
 [ 21 227  28]
 [ 25  25 238]]

first 10 test values and predictions:
0 [7.24275488e-01 2.75724212e-01 3.00065461e-07]
2 [4.40976172e-06 2.40330474e-10 9.99995590e-01]
0 [4.54

In [14]:
# looking at the top models
model_info_b_tanh = models[5]
show_above(model_info_b_tanh, 0.85)

0.9140625
	features: all features
	layers: 112
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9140625
0.9088541666666666
	features: all features
	layers: (112, 112, 112)
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.9088541666666666
0.9027777777777778
	features: all features
	layers: (112, 56, 30, 10)
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.9027777777777778
0.9010416666666666
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.9010416666666666
0.8984375
	features: all features
	layers: (112, 112, 112)
	test size: 0.4
	alpha: 0.01
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.8984375
0.8984375
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.01
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.8984375
0.8940972222222222
	features: all features
	layers: (112, 56, 30, 10)
	test size: 0.4
	alpha: 0.

### User C

In [7]:
model_info_c_log = {} # key is just an int, values are info 
automate_run(df3, model_info_c_log)



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.01



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.001



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.0001



all fea



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=1000



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=100



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=1.0



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.1



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.01



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.001



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.0001



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=1e-05



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=1000



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=100



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=1.0



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=0.1



all features:
layers=[112, 56, 30, 10]
test_size

In [16]:
# looking at the top models
# model_info_c_log = models[2]
show_above(model_info_c_log, 0.6)

0.7048611111111112
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 0.9994212962962963
	test accuracy: 0.7048611111111112
0.6666666666666666
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.1
	lambda: 0.0001
	train accuracy: 1.0
	test accuracy: 0.6666666666666666
0.6649305555555556
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.6649305555555556
0.6614583333333334
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 1.0
	lambda: 1e-05
	train accuracy: 1.0
	test accuracy: 0.6614583333333334
0.6571180555555556
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.1
	lambda: 1e-05
	train accuracy: 1.0
	test accuracy: 0.6571180555555556
0.65625
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 1.0
	lambda: 0.001
	train accuracy: 1.0
	test accuracy: 0.65625
0.6493055555555556
	features: all features
	layers: [112]
	test size

In [9]:
model_info_c_tanh = {} # key is just an int, values are info
automate_run(df3, model_info_c_tanh, act='tanh')



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.01



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.001



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=0.0001



all fea



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.1
lambda=1e-05



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=1000



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=100



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=1.0



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.1



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.01



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.001



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=0.0001



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.01
lambda=1e-05



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=1000



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=100



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=0.001
lambda=1.0



all features:
layers=[112, 56, 30, 10]
test_size

In [18]:
# looking at the top models
# model_info_c_tanh = models[6]
show_above(model_info_c_tanh, 0.6)

0.6710069444444444
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.001
	lambda: 1.0
	train accuracy: 0.9895833333333334
	test accuracy: 0.6710069444444444
0.6710069444444444
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.6710069444444444
0.6666666666666666
	features: all features
	layers: [112, 56, 30, 10]
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 0.9936342592592593
	test accuracy: 0.6666666666666666
0.6657986111111112
	features: all features
	layers: [112, 112, 112]
	test size: 0.4
	alpha: 0.1
	lambda: 0.01
	train accuracy: 1.0
	test accuracy: 0.6657986111111112
0.6631944444444444
	features: all features
	layers: [112, 112, 112]
	test size: 0.4
	alpha: 0.001
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.6631944444444444
0.6597222222222222
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 0.9976851851851852
	test accuracy: 0.65972

### User D

In [11]:
model_info_d_log = {} # key is accuracy, values are 
automate_run(df4, model_info_d_log)



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 81.33680555555556%

Confusion Matrix: 
[[311  33  50]
 [ 34 319  31]
 [ 27  40 307]]

              precision    recall  f1-score   support

           0      0.836     0.789     0.812       394
           1      0.814     0.831     0.822       384
           2      0.791     0.821     0.806       374

    accuracy                          0.813      1152
   macro avg      0.814     0.814     0.813      1152
weighted avg      0.814     0.813     0.813      1152


first 10 test values and predictions:
2 [0.06240474 0.78302597 0.1545693 ]
0 [9.98819105e-01 2.57522202e-06 1.17832013e-03]
2 [0.01981132 0.02708896 0.95309972]
2 [0.028402   0.00161709 0.96998091]
2 [0.28294597 0.5328492

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.1
lambda=0.01



all features

In [19]:
# looking at the top models
# model_info_d_log = models[3]
show_above(model_info_d_log, 0.80)

0.8133680555555556
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8133680555555556
0.8046875
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 1.0
	lambda: 0.01
	train accuracy: 1.0
	test accuracy: 0.8046875


In [13]:
model_info_d_tanh = {} # key is accuracy, values are 
automate_run(df4, model_info_d_tanh, act='tanh')



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=[112]
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=[112]
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 80.20833333333334%

Confusion Matrix: 
[[315  32  46]
 [ 21 320  43]
 [ 37  49 289]]

              precision    recall  f1-score   support

           0      0.845     0.802     0.822       393
           1      0.798     0.



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.01
lambda=0.001



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.01
lambda=0.0001



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.01
lambda=1e-05



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=1000



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=100



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=1.0



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=0.1



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=0.01



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=0.001



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=0.0001



all features:
layers=[112, 112, 112]
test_size=0.4
alpha=0.001
lambda=1e-05



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=[112, 56, 30, 10]
test_size=0.4
alpha=1

In [20]:
# looking at the top models
# model_info_d_tanh = models[7]
show_above(model_info_d_tanh, 0.8)

0.8159722222222222
	features: all features
	layers: [112, 112, 112]
	test size: 0.4
	alpha: 0.01
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.8159722222222222
0.8098958333333334
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.01
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.8098958333333334
0.8020833333333334
	features: all features
	layers: [112]
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.8020833333333334


In [20]:
len(models[0])

576

In [21]:
for x in [1,2,22,41,21,42,23,39]:
    print(x/576*100)

0.1736111111111111
0.3472222222222222
3.8194444444444446
7.118055555555555
3.6458333333333335
7.291666666666667
3.9930555555555554
6.770833333333333


## Saving the data

In [35]:
models = [
    model_info_a_log, 
    model_info_b_log,
    model_info_c_log,
    model_info_d_log,
    
    model_info_a_tanh,
    model_info_b_tanh,
    model_info_c_tanh,
    model_info_d_tanh 
]

In [7]:
# to save the model info trained here
# NOTE: this is not to save numpy arrays. If we want to do that, use np.save and np.load.
import pickle

models_file = open('models', 'wb')

pickle.dump(models, models_file)

models_file.close()

In [37]:
# to load data into models again
import pickle

models_file = open('models', 'rb')

models_loaded = pickle.load(models_file)

models_file.close()

In [40]:
models[0][1]

{'features': 'all features',
 'layers': (112, 56, 30, 10),
 'test size': 0.4,
 'alpha': 1.0,
 'lambda': 100,
 'train accuracy': 0.34664351851851855,
 'test accuracy': 0.3133680555555556,
 'classifier': MLPClassifier(activation='logistic', alpha=100,
               hidden_layer_sizes=(112, 56, 30, 10), learning_rate_init=1.0,
               max_iter=3000, random_state=1, solver='sgd')}

In [41]:
models_loaded[0][1]

{'features': 'all features',
 'layers': (112, 56, 30, 10),
 'test size': 0.4,
 'alpha': 1.0,
 'lambda': 100,
 'train accuracy': 0.34664351851851855,
 'test accuracy': 0.3133680555555556,
 'classifier': MLPClassifier(activation='logistic', alpha=100,
               hidden_layer_sizes=(112, 56, 30, 10), learning_rate_init=1.0,
               max_iter=3000, random_state=1, solver='sgd')}

In [11]:
name, X, y = 'all features', df1.values[:, 1:], df1.values[:, 0].astype(int)
# for User A (logistic)
run(X, y, [112], 0.4, 0.1, 1.0, act='logistic')


* * *
training accuracy: 99.94212962962963%
testing accuracy: 84.02777777777779%

Confusion Matrix: 
[[332  28  23]
 [ 32 305  40]
 [ 32  29 331]]

              precision    recall  f1-score   support

           0      0.838     0.867     0.852       383
           1      0.843     0.809     0.825       377
           2      0.840     0.844     0.842       392

    accuracy                          0.840      1152
   macro avg      0.840     0.840     0.840      1152
weighted avg      0.840     0.840     0.840      1152


first 10 test values and predictions:
0 [0.88274736 0.11249278 0.00475986]
1 [0.01971875 0.95776646 0.0225148 ]
2 [1.84678899e-03 2.33162849e-04 9.97920048e-01]
2 [0.00138717 0.284972   0.71364082]
0 [0.97799087 0.00282087 0.01918826]
0 [0.97066732 0.02812064 0.00121204]
1 [1.11920900e-04 9.98439333e-01 1.44874611e-03]
1 [0.23196596 0.75622065 0.01181339]
2 [0.05557761 0.15870215 0.78572025]
2 [0.00206253 0.64423229 0.35370518]


(0.9994212962962963,
 0.8402777777777778,
 MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=[112],
               learning_rate_init=1.0, max_iter=3000, random_state=1,
               solver='sgd'))

In [13]:
name, X, y = 'all features', df1.values[:, 1:], df1.values[:, 0].astype(int)
# for User A (logistic)
run(X, y, [112], 0.4, 1.0, 0.1, act='tanh')


* * *
training accuracy: 99.94212962962963%
testing accuracy: 85.85069444444444%

Confusion Matrix: 
[[332  32  21]
 [ 27 333  40]
 [ 22  21 324]]

              precision    recall  f1-score   support

           0      0.871     0.862     0.867       385
           1      0.863     0.833     0.847       400
           2      0.842     0.883     0.862       367

    accuracy                          0.859      1152
   macro avg      0.859     0.859     0.859      1152
weighted avg      0.859     0.859     0.858      1152


first 10 test values and predictions:
2 [0.30123075 0.13306128 0.56570797]
1 [0.00110642 0.96462784 0.03426574]
1 [0.11665985 0.7624231  0.12091705]
0 [0.40940016 0.0727716  0.51782824]
1 [0.70159477 0.24419896 0.05420627]
1 [0.13203504 0.84700343 0.02096153]
0 [0.7583615  0.08600734 0.15563117]
0 [0.51633301 0.41148418 0.0721828 ]
2 [0.00498611 0.04230753 0.95270636]
2 [0.02030072 0.00322702 0.97647226]


(0.9994212962962963,
 0.8585069444444444,
 MLPClassifier(activation='tanh', alpha=1.0, hidden_layer_sizes=[112],
               learning_rate_init=0.1, max_iter=3000, random_state=1,
               solver='sgd'))

In [17]:
name, X, y = 'all features', df2.values[:, 1:], df2.values[:, 0].astype(int)
# for User A (logistic)
run(X, y, [112, 112, 112], 0.4, a=0.1, lam=0.1, act='logistic')


* * *
training accuracy: 100.0%
testing accuracy: 88.10763888888889%

Confusion Matrix: 
[[334  27  16]
 [ 17 337  27]
 [ 18  32 344]]

              precision    recall  f1-score   support

           0      0.905     0.886     0.895       377
           1      0.851     0.885     0.867       381
           2      0.889     0.873     0.881       394

    accuracy                          0.881      1152
   macro avg      0.882     0.881     0.881      1152
weighted avg      0.882     0.881     0.881      1152


first 10 test values and predictions:
2 [2.10071046e-04 6.63567759e-04 9.99126361e-01]
1 [0.02262068 0.9736043  0.00377502]
1 [0.02531789 0.96074648 0.01393563]
0 [0.03000409 0.20737961 0.76261631]
2 [7.64366436e-04 1.50627283e-02 9.84172905e-01]
2 [1.64498578e-04 2.15387767e-01 7.84447734e-01]
0 [9.85080825e-01 1.48360242e-02 8.31505206e-05]
1 [4.11104089e-04 9.98992259e-01 5.96637174e-04]
2 [0.01344067 0.03943187 0.94712746]
1 [8.13811579e-06 9.99618798e-01 3.73064347e-04]


(1.0,
 0.8810763888888888,
 MLPClassifier(activation='logistic', alpha=0.1,
               hidden_layer_sizes=[112, 112, 112], learning_rate_init=0.1,
               max_iter=3000, random_state=1, solver='sgd'))

In [15]:
name, X, y = 'all features', df2.values[:, 1:], df2.values[:, 0].astype(int)
# for User A (logistic)
run(X, y, [112], 0.4, a=1.0, lam=0.1, act='tanh')


* * *
training accuracy: 98.37962962962963%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[337  27  29]
 [ 36 291  49]
 [ 24  46 313]]

              precision    recall  f1-score   support

           0      0.849     0.858     0.853       393
           1      0.799     0.774     0.786       376
           2      0.801     0.817     0.809       383

    accuracy                          0.817      1152
   macro avg      0.816     0.816     0.816      1152
weighted avg      0.817     0.817     0.817      1152


first 10 test values and predictions:
1 [8.43149044e-10 9.99999998e-01 7.35851687e-10]
1 [5.54656844e-04 9.99445339e-01 4.62021209e-09]
2 [4.36232622e-16 1.36412184e-16 1.00000000e+00]
1 [4.07927396e-03 9.95920726e-01 7.45684780e-12]
2 [8.25300881e-04 1.98281674e-05 9.99154871e-01]
1 [2.74362459e-09 9.99999961e-01 3.65639835e-08]
2 [2.24553200e-09 8.51099829e-07 9.99999147e-01]
2 [1.54250756e-07 4.42297854e-07 9.99999403e-01]
1 [4.78041273e-11 3.55590622e-04 9.99644

(0.9837962962962963,
 0.8168402777777778,
 MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=[112],
               learning_rate_init=1.0, max_iter=3000, random_state=1,
               solver='sgd'))

In [30]:
name, X, y = 'all features', df4.values[:, 1:], df4.values[:, 0].astype(int)
# for User A (logistic)
run(X, y, [112], 0.4, a=1.0, lam=0.1, act='logistic', threshold=0.6)


* * *
training accuracy: 100.0%
testing accuracy: 82.63888888888889%

Confusion Matrix: 
[[318  36  38]
 [ 27 318  29]
 [ 35  35 316]]

              precision    recall  f1-score   support

           0      0.837     0.811     0.824       392
           1      0.817     0.850     0.834       374
           2      0.825     0.819     0.822       386

    accuracy                          0.826      1152
   macro avg      0.826     0.827     0.826      1152
weighted avg      0.827     0.826     0.826      1152


first 10 test values and predictions:
1 [0.16294686 0.83549834 0.0015548 ]
0 [9.97673068e-01 1.97000832e-03 3.56923809e-04]
1 [0.00518022 0.97918205 0.01563773]
0 [9.97269988e-01 9.15134686e-04 1.81487742e-03]
0 [0.98598181 0.01226934 0.00174885]
0 [9.96887031e-01 2.47361613e-03 6.39352909e-04]
1 [1.31374260e-04 9.97521680e-01 2.34694577e-03]
0 [0.55817919 0.03549134 0.40632947]
1 [0.07930599 0.47992301 0.440771  ]
0 [0.44352421 0.29364682 0.26282896]


(1.0,
 0.8263888888888888,
 MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=[112],
               learning_rate_init=1.0, max_iter=3000, random_state=1,
               solver='sgd'))

In [32]:
name, X, y = 'all features', df4.values[:, 1:], df4.values[:, 0].astype(int)
# for User A (logistic)
run(X, y, [112], 0.4, a=0.01, lam=1.0, act='tanh', threshold=0.6)


* * *
training accuracy: 100.0%
testing accuracy: 80.38194444444444%

Confusion Matrix: 
[[305  22  42]
 [ 33 324  36]
 [ 36  57 297]]

              precision    recall  f1-score   support

           0      0.816     0.827     0.821       369
           1      0.804     0.824     0.814       393
           2      0.792     0.762     0.776       390

    accuracy                          0.804      1152
   macro avg      0.804     0.804     0.804      1152
weighted avg      0.804     0.804     0.804      1152


first 10 test values and predictions:
2 [0.36382432 0.0064411  0.62973458]
1 [0.01835852 0.56601005 0.41563143]
1 [7.08281472e-04 9.35972080e-01 6.33196382e-02]
0 [0.97009442 0.00361263 0.02629295]
1 [0.02336865 0.78588626 0.19074509]
0 [0.87744994 0.11965703 0.00289303]
1 [0.03051745 0.9673473  0.00213525]
2 [0.02310245 0.80922118 0.16767637]
2 [0.0076518  0.03259015 0.95975805]
1 [0.06495481 0.26577438 0.66927081]


(1.0,
 0.8038194444444444,
 MLPClassifier(activation='tanh', alpha=1.0, hidden_layer_sizes=[112],
               learning_rate_init=0.01, max_iter=3000, random_state=1,
               solver='sgd'))